# Skytrax Airline Review 

The dataset contains airline review collected by Skytrax airline which is  United Kingdom-based consultancy which runs an airline and airport review and ranking site. 

* Dataset Link: https://www.kaggle.com/efehandanisman/skytrax-airline-reviews

Predictive analysis is performed to analyze the data. The dataset contains feilds like Airline, Overall, Author, Review_date, Customer_review, Aircraft, Traveller_type, Cabin, Route, Date_flown, Seat_Comfort, Cabin_service, Food_bev, Entertainment, Ground_service, Value_for_money, Recommended. As the features comprise of numerical data Linear Regression is used to predict the predictors.

* Importing Spark building Spark session and then Loading the data into data frame.

In [1]:
import pyspark 
from pyspark.sql import SparkSession 
#SparkSession is now the entry point of Spark 
#SparkSession can also be construed as gateway to spark libraries 

#creating instance of spark class 
spark=SparkSession.builder.appName('skytrax').getOrCreate() 

#creating spark dataframe from input csv file 
df=spark.read.csv("V:\IOS\Big Data Analytics\Technical Final\cleann_capstone_airline_reviews.csv"
				,inferSchema=True,header=True) 
df.show(10) 


+----------------+-------+--------------------+---------------+--------------------+--------------------+--------------+--------------+--------------------+-------------+------------+-------------+--------+-------------+--------------+---------------+-----------+
|         airline|overall|              author|    review_date|     customer_review|            aircraft|traveller_type|         cabin|               route|   date_flown|seat_comfort|cabin_service|food_bev|entertainment|ground_service|value_for_money|recommended|
+----------------+-------+--------------------+---------------+--------------------+--------------------+--------------+--------------+--------------------+-------------+------------+-------------+--------+-------------+--------------+---------------+-----------+
|Turkish Airlines|     10|         Zeshan Shah|   6th May 2019|Ã¢Å“â€¦ Trip Veri...|                A330|  Solo Leisure| Economy Class|Washington Dulles...|   April 2019|           4|            5|       5|  

* Now check for the Schema of the loaded dataframe as some columns Numeric Columns are stored in string format.

In [2]:
#prints structure of dataframe along with datatype 
df.printSchema() 

root
 |-- airline: string (nullable = true)
 |-- overall: integer (nullable = true)
 |-- author: string (nullable = true)
 |-- review_date: string (nullable = true)
 |-- customer_review: string (nullable = true)
 |-- aircraft: string (nullable = true)
 |-- traveller_type: string (nullable = true)
 |-- cabin: string (nullable = true)
 |-- route: string (nullable = true)
 |-- date_flown: string (nullable = true)
 |-- seat_comfort: string (nullable = true)
 |-- cabin_service: string (nullable = true)
 |-- food_bev: string (nullable = true)
 |-- entertainment: string (nullable = true)
 |-- ground_service: string (nullable = true)
 |-- value_for_money: string (nullable = true)
 |-- recommended: string (nullable = true)



* Converting the Seat_comfort, Cabin_service, Foob_bev, entertainment, Value_for_money, Ground_service, Recommended which are numerical but stored in a string format. 
* By Importing Pyspark Sql cast the required feilds into intergers from string type.

In [3]:
from pyspark.sql.types import IntegerType
df = df.withColumn("seat_comfort", df.seat_comfort.cast('integer'))
df = df.withColumn("cabin_service", df.cabin_service.cast('integer'))
df = df.withColumn("food_bev", df.food_bev.cast('integer'))
df = df.withColumn("entertainment", df.entertainment.cast('integer'))
df = df.withColumn("value_for_money", df.value_for_money.cast('integer'))
df = df.withColumn("ground_service", df.ground_service.cast('integer'))
df = df.withColumn("recommended", df.ground_service.cast('integer'))

* Check for the Schema for all columns in the dataframe the schema of casted columns are changed into interger.

In [4]:
df.printSchema() 

root
 |-- airline: string (nullable = true)
 |-- overall: integer (nullable = true)
 |-- author: string (nullable = true)
 |-- review_date: string (nullable = true)
 |-- customer_review: string (nullable = true)
 |-- aircraft: string (nullable = true)
 |-- traveller_type: string (nullable = true)
 |-- cabin: string (nullable = true)
 |-- route: string (nullable = true)
 |-- date_flown: string (nullable = true)
 |-- seat_comfort: integer (nullable = true)
 |-- cabin_service: integer (nullable = true)
 |-- food_bev: integer (nullable = true)
 |-- entertainment: integer (nullable = true)
 |-- ground_service: integer (nullable = true)
 |-- value_for_money: integer (nullable = true)
 |-- recommended: integer (nullable = true)



* Calling all the available columns in the dataframe.

In [5]:
#In our predictive model, below are the columns 
df.columns 

['airline',
 'overall',
 'author',
 'review_date',
 'customer_review',
 'aircraft',
 'traveller_type',
 'cabin',
 'route',
 'date_flown',
 'seat_comfort',
 'cabin_service',
 'food_bev',
 'entertainment',
 'ground_service',
 'value_for_money',
 'recommended']

* Feature selection: The required Fetaures or the independent varibales have to be selected for the model.
* Aircraft Feild is in String type But it is an feature required for Model. As the linear regression is used for the model. Aircraft feild which contains name of type of Aircraft Like boeing 777 has to be converted into Numeric datatype.
* By using String indexer the value is can be changed into numreic value and new columns is added to the data frame with the column name 'aircraft_cat' which contains Numeric datatype of aircraft.
* And then displaying the first five rows to check for the new column of aircraft_cat is added to the dataframe.

In [6]:
#columns identified as features are as below: 
#['seat_comfort', 'cabin_service', 'food_bev', 'entertainment', 'ground_service', 'aircraft'] 
#to work on the features, spark MLlib expects every value to be in numeric form 
#feature 'aircraft' is string datatype 
#using StringIndexer, string type will be typecast to numeric datatype 
#import library strinindexer for typecasting 

from pyspark.ml.feature import StringIndexer 
indexer=StringIndexer(inputCol='aircraft',outputCol='aircraft_cat') 
indexed=indexer.fit(df).transform(df) 

#above code will convert string to numeric feature and create a new dataframe 
#new dataframe contains a new feature 'aircraft_cat' and can be used further 
#feature aircraft_cat is now vectorized and can be used to fed to model 
for item in indexed.head(5): 
	print(item) 
	print('\n') 


Row(airline='Turkish Airlines', overall=10, author='Zeshan Shah', review_date='6th May 2019', customer_review='Ã¢Å“â€¦ Trip Verified | Flew on Turkish Airlines IAD-IST-KHI and return KHI-IST-IAD. Turkish Airlines has consistently maintained its quality since I first flew with them in 2007. The flights leave on time, the catering is excellent, the inflight entertainment is extensive and the interface easy to use, and the cabin crew is excellent. Interesting though the A330 on the KHI-IST route and return seemed to have more leg room and was newer than the A330 on the IAD-IST route which was showing its age. The A330 on the IAD-IST route had a slow responding interface for the inflight entertainment and a broken table on the return flight. But Turkish Airlines will be replacing the A330 on its flight to IAD with the 787 sometime in the summer. Turkish food was served on the return leg which I personally like, and I saw the cabin staff helping elderly passengers walk to the lavatory which

# Model for Predicting Overall Rating

* Importing vector and Vector assembler as the MLlib takes input in vector form. So the selected Features are converted into vector form using Vector assembler
* Features are transformed into vectors and Overall Rating column is selected for predictions. Features and Overall columns are displayed.

In [7]:
from pyspark.ml.linalg import Vectors 
from pyspark.ml.feature import VectorAssembler 
#creating vectors from features 
#Apache MLlib takes input if vector form 
assembler=VectorAssembler(inputCols=[ 'seat_comfort', 
'cabin_service', 
'food_bev', 
'entertainment', 
'ground_service', 
'aircraft_cat'],outputCol='features') 
assembler.setHandleInvalid("skip").transform(indexed).show
output=assembler.transform(indexed) 
output.select('features','overall').show(5) 
#output as below 

+--------------------+-------+
|            features|overall|
+--------------------+-------+
|[4.0,5.0,5.0,5.0,...|     10|
|[3.0,3.0,2.0,3.0,...|      2|
|[2.0,3.0,3.0,3.0,...|      6|
|[1.0,1.0,1.0,1.0,...|      1|
|[1.0,3.0,2.0,2.0,...|      2|
+--------------------+-------+
only showing top 5 rows



* Selecting features and Label which is Overall into final_data

In [8]:
#final data consist of features and label which is Overall. 
final_data=output.select('features','overall')

* Displaying the head of the Final_data. Each row contains Selected features in a vector and along with the Overall Rating.

In [9]:
final_data.head()

Row(features=DenseVector([4.0, 5.0, 5.0, 5.0, 5.0, 3.0]), overall=10)

* Randomly Split the dataset into Train and test data. This is should be done using Random split because If the data in dataset is sequential data then the efficiency and preditictions will be not accurate to the original values.
* Then displaying number of rows in train and test data And then statistics of train_data.

In [10]:
#splitting data into train and test 
train_data ,test_data = final_data.randomSplit([0.7,0.3]) 
print(train_data.count())
print(test_data.count())
train_data.describe().show() 

12703
5465
+-------+-----------------+
|summary|          overall|
+-------+-----------------+
|  count|            12703|
|   mean|6.468393292922932|
| stddev|3.086240866867872|
|    min|                1|
|    max|               10|
+-------+-----------------+



* Importing Linear Regression and applying it to the Features and Overall label as model.

In [11]:
#import LinearRegression library 
from pyspark.ml.regression import LinearRegression 
#creating an object of class LinearRegression
#object takes features and label as input arguments 
model=LinearRegression(featuresCol='features',labelCol='overall')

* Fitting the model. The fit finds the best fit for the model.
* Evaluting the train data which shows the metrics like R square error.

In [12]:
#pass train_data to train model 
trained_model=model.fit(train_data) 
#evaluating model trained for Rsquared error 
overall_results=trained_model.evaluate(train_data) 

* Calculating the R Square value. The R square value close to the is the best value and describe realtion of model with the features.

In [13]:
# R-squared
print('Rsquared Error :',overall_results.r2)

Rsquared Error : 0.8084649321207266


* Testing the model with the unseen data.
* Displaying First 5 rows in test data.

In [14]:
#testing Model on unlabeled data 
#create unlabeled data from test_data 
#testing model on unlabeled data 
unlabeled_data=test_data.select('features') 
unlabeled_data.show(5) 

+--------------------+
|            features|
+--------------------+
|[1.0,1.0,1.0,1.0,...|
|[1.0,1.0,1.0,1.0,...|
|[1.0,1.0,1.0,1.0,...|
|[1.0,1.0,1.0,1.0,...|
|[1.0,1.0,1.0,1.0,...|
+--------------------+
only showing top 5 rows



* Predicting the overall rating from the test_data features.

In [15]:
predictions=trained_model.transform(unlabeled_data) 
predictions.show()
#below are the results of output from test data 

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[1.0,1.0,1.0,1.0,...|0.5957534173976191|
|[1.0,1.0,1.0,1.0,...|0.5957896828190867|
|[1.0,1.0,1.0,1.0,...|0.5957896828190867|
|[1.0,1.0,1.0,1.0,...|0.5957896828190867|
|[1.0,1.0,1.0,1.0,...|0.5957896828190867|
|[1.0,1.0,1.0,1.0,...|0.5958984790834896|
|[1.0,1.0,1.0,1.0,...|0.5958984790834896|
|[1.0,1.0,1.0,1.0,...|0.5960072753478929|
|[1.0,1.0,1.0,1.0,...|0.5960435407693605|
|[1.0,1.0,1.0,1.0,...|0.5960435407693605|
|[1.0,1.0,1.0,1.0,...|0.5960798061908281|
|[1.0,1.0,1.0,1.0,...|0.5960798061908281|
|[1.0,1.0,1.0,1.0,...|0.5960798061908281|
|[1.0,1.0,1.0,1.0,...|0.5960798061908281|
|[1.0,1.0,1.0,1.0,...|0.5960798061908281|
|[1.0,1.0,1.0,1.0,...|0.5960798061908281|
|[1.0,1.0,1.0,1.0,...|0.5960798061908281|
|[1.0,1.0,1.0,1.0,...|0.5961160716122957|
|[1.0,1.0,1.0,1.0,...| 0.596188602455231|
|[1.0,1.0,1.0,1.0,...| 0.596188602455231|
+--------------------+------------

* Importing Regression metrics and calculating the metrics of the model.

In [16]:
# Squared Error
from pyspark.mllib.evaluation import RegressionMetrics
print("MSE = %s" % overall_results.meanSquaredError)
print("RMSE = %s" % overall_results.rootMeanSquaredError)

MSE = 1.8242054366437856
RMSE = 1.3506314955026726


In [17]:
# Mean absolute error
print("MAE = %s" % overall_results.meanAbsoluteError)

# Explained variance
print("Explained variance = %s" % overall_results.explainedVariance)

MAE = 0.9693644370694899
Explained variance = 7.6999274380400085


* The above Metrics Results shows that model accuracy is very good and can be use for predictive analysis.

# Model for predicting recommandations

* Importing vector and Vector assembler as the MLlib takes input in vector form. So the selected Features are converted into vector form using Vector assembler
* Features are transformed into vectors and Recommended column is selected for predictions. Features and Overall columns are displayed.

In [18]:
from pyspark.ml.linalg import Vectors 
from pyspark.ml.feature import VectorAssembler 
#creating vectors from features 
#Apache MLlib takes input if vector form 
assembler=VectorAssembler(inputCols=[ 'seat_comfort', 
'cabin_service', 
'food_bev', 
'entertainment', 
'ground_service'],outputCol='features') 
assembler.setHandleInvalid("skip").transform(df).show
output=assembler.transform(df) 
output.select('features','recommended')

DataFrame[features: vector, recommended: int]

* Selecting features and Label which is Overall into final_data

In [19]:
#final data consist of features and label which is Recommended. 
final_data=output.select('features','recommended')

* Randomly Split the dataset into Train and test data. This is should be done using Random split because If the data in dataset is sequential data then the efficiency and preditictions will be not accurate to the original values.
* Then displaying number of rows in train and test data And then statistics of train_data.

In [20]:
#splitting data into train and test 
train_data ,test_data = final_data.randomSplit([0.7,0.3]) 
print(train_data.count())
print(test_data.count())
train_data.describe().show() 

12737
5431
+-------+-----------------+
|summary|      recommended|
+-------+-----------------+
|  count|            12737|
|   mean| 3.44241187092722|
| stddev|1.435320586993376|
|    min|                1|
|    max|                5|
+-------+-----------------+



* Importing Linear Regression and applying it to the Features and Recommended label as model.

In [21]:
#import LinearRegression library 
from pyspark.ml.regression import LinearRegression 
#creating an object of class LinearRegression 
#object takes features and label as input arguments 
model=LinearRegression(featuresCol='features',labelCol='recommended')

* Fitting the model. The fit finds the best fit for the model.
* Evaluting the train data which shows the metrics like R square error.

In [22]:
#pass train_data to train model 
trained_model=model.fit(train_data) 
#evaluating model trained for Rsquared error 
recommended_results=trained_model.evaluate(train_data) 

* Calculating the R Square value. The R square value close to the is the best value and describe realtion of model with the features.
* The model has the R-sqaure value of 1 which shows the deep relation of features with the label and best model for predicting the label based on features.

In [23]:
# R-squared
print('Rsquared Error :',recommended_results.r2)

Rsquared Error : 1.0


* Testing the model with the unseen data.
* Displaying First 5 rows in test data.

In [24]:
#testing Model on unlabeled data 
#create unlabeled data from test_data 
#testing model on unlabeled data 
unlabeled_data=test_data.select('features') 
unlabeled_data.show(5) 

+--------------------+
|            features|
+--------------------+
|[1.0,1.0,1.0,1.0,...|
|[1.0,1.0,1.0,1.0,...|
|[1.0,1.0,1.0,1.0,...|
|[1.0,1.0,1.0,1.0,...|
|[1.0,1.0,1.0,1.0,...|
+--------------------+
only showing top 5 rows



* Predicting the overall rating from the test_data features.

In [25]:
predictions=trained_model.transform(unlabeled_data) 
predictions.show()
#below are the results of output from test data 

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[1.0,1.0,1.0,1.0,...|0.9999999999999997|
|[1.0,1.0,1.0,1.0,...|0.9999999999999997|
|[1.0,1.0,1.0,1.0,...|0.9999999999999997|
|[1.0,1.0,1.0,1.0,...|0.9999999999999997|
|[1.0,1.0,1.0,1.0,...|0.9999999999999997|
|[1.0,1.0,1.0,1.0,...|0.9999999999999997|
|[1.0,1.0,1.0,1.0,...|0.9999999999999997|
|[1.0,1.0,1.0,1.0,...|0.9999999999999997|
|[1.0,1.0,1.0,1.0,...|0.9999999999999997|
|[1.0,1.0,1.0,1.0,...|0.9999999999999997|
|[1.0,1.0,1.0,1.0,...|0.9999999999999997|
|[1.0,1.0,1.0,1.0,...|0.9999999999999997|
|[1.0,1.0,1.0,1.0,...|0.9999999999999997|
|[1.0,1.0,1.0,1.0,...|0.9999999999999997|
|[1.0,1.0,1.0,1.0,...|0.9999999999999997|
|[1.0,1.0,1.0,1.0,...|0.9999999999999997|
|[1.0,1.0,1.0,1.0,...|0.9999999999999997|
|[1.0,1.0,1.0,1.0,...|0.9999999999999997|
|[1.0,1.0,1.0,1.0,...|0.9999999999999997|
|[1.0,1.0,1.0,1.0,...|0.9999999999999997|
+--------------------+------------

* Importing Regression metrics and calculating the metrics of the model.

In [26]:
# Squared Error
from pyspark.mllib.evaluation import RegressionMetrics
print("MSE = %s" % overall_results.meanSquaredError)
print("RMSE = %s" % overall_results.rootMeanSquaredError)

MSE = 1.8242054366437856
RMSE = 1.3506314955026726


In [27]:
# Mean absolute error
print("MAE = %s" % overall_results.meanAbsoluteError)

# Explained variance
print("Explained variance = %s" % overall_results.explainedVariance)

MAE = 0.9693644370694899
Explained variance = 7.6999274380400085


* The above Metrics Results shows that model accuracy is very good and can be use for predictive analysis.
* The R square value of the model is 1 which the model is best model for the predictions and results best fit.

* Terminating the Spark session

In [29]:
spark.stop()